## a. カメラパラメータ計算

In [1]:
import cv2
import numpy as np
import pandas as pd

world_pts = np.float32([
  [10,0,10], # 1
  [90,0,10], # 3
  [10,0,90], # 5
  [90,0,90], # 7
  [0,10,10], # 2
  [0,90,10], # 4
  [0,10,90], # 6
  [0,90,90], # 8
  [10,10,100], # 9
  [90,10,100], # 10
  [10,90,100], # 11
  [90,90,100], # 12
])

# === 2. ここに対応する (u,v) pxを記入===
# plot_image_xy.py で表示される画像の座標を入力してください。
img_pts = np.float32([
  [1378,1303],
  [1698,982],
  [1376,782],
  [1724,489],
  [1263,1311],
  [783,1086],
  [1255,797],
  [736, 588],
  [1304, 676],
  [1664, 391],
  [774, 482],
  [1155,230],
])

h, w = 1520, 2288     # 画像サイズ(px)を入力
K0 = np.array([[w,0,w/2],[0,w, h/2],[0,0,1]], dtype=np.float64)  # 粗い初期値

ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
    [world_pts], [img_pts], (w,h),
    K0, None,
    flags=cv2.CALIB_USE_INTRINSIC_GUESS |
          cv2.CALIB_ZERO_TANGENT_DIST |
          cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5)

# CSV に保存して終了
pd.DataFrame(np.hstack([K, np.zeros((3,1))])).to_csv("kadai1/K.csv", index=False, header=False)
pd.DataFrame(cv2.Rodrigues(rvecs[0])[0]).to_csv("kadai1/R.csv", index=False, header=False)
pd.DataFrame(tvecs[0]).to_csv("kadai1/t.csv", index=False, header=False)
print("RMS reprojection error:", ret)


RMS reprojection error: 3.2737715885582848


### b. x, y座標計算

In [7]:
import numpy as np
import pandas as pd

K = pd.read_csv('kadai1/K.csv', header=None).values
R = pd.read_csv('kadai1/R.csv', header=None).values
t = pd.read_csv('kadai1/t.csv', header=None).values.reshape(-1)

fx, fy, cx, cy = K[0,0], K[1,1], K[0,2], K[1,2]
Z0 = 70.0           # mm

def solve_xy(u, v):
    a = (u - cx)/fx; b = (v - cy)/fy         # α, β
    A1 = a*R[2,0] - R[0,0];  B1 = a*R[2,1] - R[0,1]
    C1 = R[0,2]*Z0 + t[0] - a*(R[2,2]*Z0 + t[2])
    A2 = b*R[2,0] - R[1,0];  B2 = b*R[2,1] - R[1,1]
    C2 = R[1,2]*Z0 + t[1] - b*(R[2,2]*Z0 + t[2])
    XY = np.linalg.solve([[A1,B1],[A2,B2]], [C1,C2])
    return XY  # [X, Y]

# 例: (u,v) = (600,320) の交点
# x側の交点（理論上yはゼロなはず）
print("x側の交点")
print("交点1: ", solve_xy(1378, 924))    
print("交点2: ", solve_xy(1426, 880))    
print("交点3: ", solve_xy(1473,843))    
print("交点4: ", solve_xy(1513,796))    
print("交点5: ", solve_xy(1551,760))
print("交点6: ", solve_xy(1595,726))
print("交点7: ", solve_xy(1636,689))
print("交点8: ", solve_xy(1682,656))
print("交点9: ", solve_xy(1716, 623))

# y側の交点（理論上xはゼロなはず）
print("y側の交点")
print("交点10: ", solve_xy(1258,936))
print("交点11:", solve_xy(1188,905))
print("交点12:", solve_xy(1116,872))
print("交点13:", solve_xy(1063,849))
print("交点14:", solve_xy(991,823))
print("交点15:", solve_xy(932,802))
print("交点16:", solve_xy(868,771))
print("交点17:", solve_xy(806,747))
print("交点18:", solve_xy(747,719))


x側の交点
交点1:  [ 9.66479698 -1.59245753]
交点2:  [20.51384092 -1.46321852]
交点3:  [30.34121959 -2.029498  ]
交点4:  [41.79139837 -0.72470038]
交点5:  [51.33444792 -0.47028063]
交点6:  [61.14489247 -1.08633004]
交点7:  [71.58532694 -1.03239016]
交点8:  [81.81389681 -2.01007653]
交点9:  [91.39735281 -1.68890734]
y側の交点
交点10:  [-0.48539194  8.98800832]
交点11: [-0.11011984 19.63662546]
交点12: [ 0.47440186 30.99805497]
交点13: [ 0.69293802 39.3822839 ]
交点14: [ 0.08740485 50.31121311]
交点15: [-0.47169855 59.398612  ]
交点16: [ 0.35915261 70.62399468]
交点17: [ 0.10573161 80.86144057]
交点18: [ 0.79074664 91.5989673 ]


### c. 歪み考察

In [3]:
import numpy as np
import pandas as pd

K = pd.read_csv('kadai1/K.csv', header=None).values
R = pd.read_csv('kadai1/R.csv', header=None).values
t = pd.read_csv('kadai1/t.csv', header=None).values.reshape(-1)

fx, fy, cx, cy = K[0,0], K[1,1], K[0,2], K[1,2]
Z0 = 10.0           # mm

def solve_xy(u, v):
    a = (u - cx)/fx; b = (v - cy)/fy         # α, β
    A1 = a*R[2,0] - R[0,0];  B1 = a*R[2,1] - R[0,1]
    C1 = R[0,2]*Z0 + t[0] - a*(R[2,2]*Z0 + t[2])
    A2 = b*R[2,0] - R[1,0];  B2 = b*R[2,1] - R[1,1]
    C2 = R[1,2]*Z0 + t[1] - b*(R[2,2]*Z0 + t[2])
    XY = np.linalg.solve([[A1,B1],[A2,B2]], [C1,C2])
    return XY  # [X, Y]

# 例: (u,v) = (600,320) の交点
# x側の交点（理論上yはゼロなはず）
print("x側の交点")
print("交点1: ", solve_xy(1376,1303))    
print("交点2: ", solve_xy(1422,1257))    
print("交点3: ", solve_xy(1466,1218))    
print("交点4: ", solve_xy(1503,1173))    
print("交点5: ", solve_xy(1544,1136))
print("交点6: ", solve_xy(1586,1098))
print("交点7: ", solve_xy(1621,1055))
print("交点8: ", solve_xy(1660,1017))
print("交点9: ", solve_xy(1694,985))

# y側の交点（理論上xはゼロなはず）
print("y側の交点")
print("交点10:", solve_xy(1265,1314))
print("交点11:", solve_xy(1199,1285))
print("交点12:", solve_xy(1136,1252))
print("交点13:", solve_xy(1075,1221))
print("交点14:", solve_xy(1012,1194))
print("交点15:", solve_xy(955,1167))
print("交点16:", solve_xy(896,1142))
print("交点17:", solve_xy(839,1112))
print("交点18:", solve_xy(787,1084))


x側の交点
交点1:  [ 8.62016068 -2.76443984]
交点2:  [19.53615369 -2.77006263]
交点3:  [29.39875586 -3.31771812]
交点4:  [40.0131136  -2.45065892]
交点5:  [49.7709502  -2.89683854]
交点6:  [60.01173339 -3.35332084]
交点7:  [70.90622525 -2.45455233]
交点8:  [81.41880697 -2.57125536]
交点9:  [90.55913193 -2.80664636]
y側の交点
交点10: [-1.31985584  7.81185867]
交点11: [-1.63978037 18.05858181]
交点12: [-1.09885159 28.65465402]
交点13: [-0.71809669 39.02650899]
交点14: [-1.13904317 49.33764047]
交点15: [-1.10207655 59.17356504]
交点16: [-1.55478336 69.15508861]
交点17: [-1.01575448 79.78769599]
交点18: [-0.40618978 89.77905375]
